In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import pandas as pd
import time

For Mac Users

In [ ]:
# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

In [ ]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

For Windows Users

In [2]:
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

## NASA Mars News

In [3]:
# URL of page to be scraped
mars_news = 'https://mars.nasa.gov/news/'
browser.visit(mars_news)
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html,'lxml')
# Use select_one to find the first tag that matches the defined selector
latest = soup.select_one("ul.item_list li.slide")

In [4]:
# Collect and store latest news title
latest_title = latest.find(class_='content_title').get_text()
# Collect and store latest news description
latest_blurb = latest.find(class_='rollover_description_inner').get_text()

## Mars Space Images - Featured Image

In [5]:
# URL of page to be scraped
mars_image = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(mars_image)
# Click the 'Full Image' button on the main page
browser.click_link_by_partial_text('FULL IMAGE')
# Allow 3 seconds so that page can load before next click
time.sleep(3)
# Click the 'Full Image' button on the main page
browser.click_link_by_partial_text('more info')
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')
# Use find() to navigate and retrieve the image url
image_url = soup.find(class_='lede').find('a')['href']
# Save image URL
featured_image_url = 'https://www.jpl.nasa.gov' + image_url

## Mars Weather

In [6]:
# URL of page to be scraped
mars_weather = 'https://twitter.com/marswxreport?lang=en'
# Retrieve page with the requests module
response = requests.get(mars_weather)
# Create BeautifulSoup object; parse with 'lxml'
soup = BeautifulSoup(response.text, 'lxml')

In [7]:
# Collect and store latest tweet
latest_tweet = soup.find(class_='js-tweet-text-container').find("p").text

## Mars Facts

In [7]:
# URL of page to be scraped
mars_facts = 'https://space-facts.com/mars/'
# Use read_html to parse the URL for tables
table_list = pd.read_html(mars_facts)
# Select the desired table from the list
mars_facts_table = table_list[1]
# Rename the columns accordingly
mars_facts_table.columns = ['Planet Profile',' ']
# Reset the index
mars_facts_table.set_index('Planet Profile',inplace=True)
# Use Pandas to convert the data to a HTML table string
html_table = mars_facts_table.to_html()

In [8]:
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th></th>\n    </tr>\n    <tr>\n      <th>Planet Profile</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\n 

## Mars Hemispheres

In [9]:
# URL of page to be scraped
mars_hemispheres = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
browser.visit(mars_hemispheres)
# HTML object
html = browser.html
# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html, 'html.parser')
# Retrieve all elements that contain hemisphere information
desc_list = soup.find_all(class_='description')

In [10]:
# Use a list comprehension to extract the name of each hemisphere
hemisphere_names = [desc.find('h3').text.replace(' Enhanced','') for desc in desc_list]
# Print to ensure all four hemisphere names were extracted
print(hemisphere_names)

['Cerberus Hemisphere', 'Schiaparelli Hemisphere', 'Syrtis Major Hemisphere', 'Valles Marineris Hemisphere']


In [11]:
# Create an empty list to contain all hemisphere dictionaries
hemisphere_image_urls  = []

# Create a for-loop that retrieves each hemisphere name and its corresponding image and inserts them into a dictionary
for name in hemisphere_names:
    # URL of the page to be scraped
    mars_hemispheres = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'
    browser.visit(mars_hemispheres)
    # Click on the name of each hemisphere to retrieve the full image
    browser.click_link_by_partial_text(f'{name} Enhanced')
    # HTML object
    html = browser.html
    # Parse HTML with Beautiful Soup
    soup = BeautifulSoup(html, 'html.parser')
    # Use find() to locate the class where the image is located
    image_class = soup.find(class_='downloads')
    # Use find() to navigate and retrieve the image url
    image_url = image_class.find('li').find('a')['href']
    # Create a dictionary containing the title and image url for each hemisphere and append to the empty list 
    hemisphere_image_urls.append({'title':name, 'img_url':image_url})